In [23]:
from PIL import Image # used for loading images
import numpy as np
import os # used for navigating to image path
import imageio # used for writing images
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from timeit import default_timer as timer
from tensorflow.keras import backend as K

"""HP Tuning"""
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

from tensorflow import keras

import IPython

from sklearn.preprocessing import LabelEncoder

import kerastuner as kt
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection

In [24]:
"""Load numpy output files"""
pr_im_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Im.npy', allow_pickle=True)
pr_po_im_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Po_Im.npy', allow_pickle=True)
pr_poim_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_PoIm.npy', allow_pickle=True)
prpo_im_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_PrPo_Im.npy', allow_pickle=True)

In [25]:
NUM_CLASS = 2
NUM_CHANNEL = 1
CLASSIFICATION_SCENARIO = "B"

In [26]:
def splitData(image_array, prop = 0.80, seed_num = 111):
    """Returns training and test arrays of images with specified proportion - prop:1-prop"""
    random.Random(seed_num).shuffle(image_array)
    train_size = int(prop*np.shape(image_array)[0])
    train = image_array[:train_size]
    test = image_array[train_size:]
    return(train, test)

In [27]:
train_data, test_data = splitData(pr_im_64, seed_num = 111)

In [28]:
def getImageShape(image_array):
    if NUM_CHANNEL==1:
        image_shape = np.array([np.expand_dims(x[0],axis=2) for x in image_array]).shape[1:4]
    elif NUM_CHANNEL==3:
        image_shape = np.array([x[0] for x in image_array]).shape[1:4][::-1]
    print(image_shape)
    return image_shape

In [29]:
input_image_shape = getImageShape(train_data)

In [30]:
# Train the model.
if NUM_CHANNEL == 1:
    train_array = np.array([np.expand_dims(x[0],axis=2) for x in train_data])
    validation_array = np.array([np.expand_dims(x[0],axis=2) for x in test_data])
elif NUM_CHANNEL == 3:
    train_array = np.array([x[0] for x in train_data]) 
    train_array = np.moveaxis(train_array, 1, -1)
    validation_array = np.array([x[0] for x in test_data])
    validation_array = np.moveaxis(validation_array, 1, -1)

train_labels = np.array([x[1] for x in train_data])
validation_labels = np.array([x[1] for x in test_data])

In [31]:
def model_builder(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

In [32]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt_Pr_Im')

In [33]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [34]:
tuner.search(train_array, train_labels, epochs = 10, validation_data = (validation_array, validation_labels), callbacks = [ClearTrainingOutput()])

In [35]:
#Return best hyperparameter dictionary
best_hps_dict = tuner.get_best_hyperparameters()[0].values
best_hps_dict

In [36]:
#model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
model_pr_im = tuner.hypermodel.build(best_hps)
start = timer()

model_pr_im.fit(train_array, train_labels, epochs=10, validation_data=(validation_array, validation_labels))

end = timer()
print(end-start)

In [37]:
model_pr_im.summary()
model_path = '../../results/models/optimized_scenario_' + CLASSIFICATION_SCENARIO
model_pr_im.save(model_path)

In [38]:
#Best Hperparamter summary
best_hps_summary = f"""
The hyperparameter search for classification scenario {CLASSIFICATION_SCENARIO} is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}. The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. The optimal kernel size of the first convolution layer is {best_hps.get('kernel_size')}.
The optimal dropout rate for the optimizer is {best_hps.get('dropout')}. The optimal activation layer for the optimizer is {best_hps.get('activation')}.
"""

best_hps_summary_path = model_path + '/best_hyperparameters_summary_scenario_' + CLASSIFICATION_SCENARIO + '.txt'
print(best_hps_summary,  file=open(best_hps_summary_path, 'w'))